In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
tf.__version__

In [0]:
"""
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
"""

In [0]:
import pathlib
data_root = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Dhivya_Classifier/videos/")

In [0]:
for item in data_root.iterdir():
  print(item)

Create an array of filenames of type string and convert to .jpg

In [0]:
import random
import os, glob
all_image_paths = list(data_root.glob('*/*'))

fileArr = []
file = "/content/drive/My Drive/Colab Notebooks/Dhivya_Classifier/videos/Video_27/RGB/147/"
for filename in glob.glob(os.path.join(file, '*.jpg')):
  fileArr.append(filename)
  
fileArr.sort()
print(fileArr)

Import data stored in google drive, creating "colA" and "colB" to store files and labels respectively

In [0]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dhivya_Classifier/data.csv", names=['colA', 'colB'], error_bad_lines = False)
data.head

Create "colC" to contain the entire file path: "/content/drive/My Drive/Colab Notebooks/Dhivya_Classifier/videos/Video_#/RGB/#/thumb00000#"

In [0]:
import os
l = []
for index, row in data.iterrows():
  l.append(os.path.join("/content/drive/My Drive/Colab Notebooks/Dhivya_Classifier/videos/",row.colA))
data['colC'] = l
data

Retrieve the current working directory (cwd) and get all the files in that directory

In [18]:
import os

cwd = os.getcwd()
files = os.listdir(cwd)
print("Files in '%s': %s" % (cwd, files))

Files in '/content': ['.config', 'drive', 'sample_data']


Count the number of image paths or video files

In [0]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

In [0]:
import IPython.display as display

def caption_image(image_path):
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    return "Image (CC BY 2.0) " + ' - '.join(attributions[str(image_rel)].split(' - ')[:-1])

Create "colD" to contain only the .jpg files

In [0]:
import numpy as np
from os import listdir
from os.path import isfile, join
onlyfiles = []
bigList = []
for index, row in data.iterrows():
    onlyfiles = [f for f in listdir(row['colC']) if isfile(join(row['colC'], f))]
    onlyfiles.append(all_image_paths)
    bigList.append(onlyfiles)
data['colD'] = bigList
print(data.shape)
print(np.array(bigList).shape)
print(onlyfiles)

Create array "bigList" to contain the entire filepath of all .jpg files and copy that data to data['colD']

In [0]:
import numpy as np
from os import listdir
from os.path import isfile, join
onlyfiles = []
bigList = []
for index, row in data.iterrows():
  tempList = []
  onlyfiles = [f for f in listdir(row['colC']) if isfile(join(row['colC'], f))]
  for i in onlyfiles:
    tempList.append(os.path.join(row['colC'], i))
  tempList.sort()
  bigList.append(tempList)
data['colD'] = bigList
print(data.shape)
print(np.array(bigList).shape)
bigList

In [0]:
print(bigList[0])

In [0]:
for index in bigList:
  for i in index:
    img_path = i
    img_raw = tf.io.read_file(img_path)
    img_final = tf.image.decode_image(img_raw)
    img_final = tf.image.convert_image_dtype(img_final, tf.float32)

print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())
print(index)
  


Read in each file in bigList and resize each image, and append to the corresponding labels (located in "colB")

In [0]:
from tqdm import tqdm
bigList = []
y = []
for index, row in tqdm(data.iterrows()):
  tempE = []
  for i in row['colD']:
    img_path = i
    img_raw = tf.io.read_file(img_path)
    img_final = tf.image.decode_image(img_raw)
    img_final = tf.image.convert_image_dtype(img_final, tf.float32)
    img_final = tf.image.resize(img_final, [20, 20])
    tempE.append(img_final)
  try:
    bigList.append(tf.convert_to_tensor(tempE))
    y.append(np.array(row['colB']))
  except:
    continue
bigList

In [0]:
bigList = tf.convert_to_tensor(bigList)
bigList.shape

In [0]:
print(data['colD'])

Currently there are 4 columns A-D, and the following code goes through the process of condensing 4 columns to 1 "colA" which will thne contain the entire filepath as an .npz file

Data is imported again to override any changes made earlier to columns A and B

In [0]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/Dhivya_Classifier/data.csv", names=['colA', 'colB'], error_bad_lines = False)
data

In [0]:
filename = data['colA']
label = data['colB']
def _parse_function(filename, label):
  image_string = tf.io.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_resized = tf.image.resize(image_decoded, [28, 28])
  return image_resized, label

# A vector of filenames.
filenames = tf.constant(data['colC'])

# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant(data['colB'])

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)

In [0]:
import os
l = []
for index, row in data.iterrows():
  l.append(os.path.join("/content/drive/My Drive/Dhivya_Classifier/videos/",row.colA))
l

In [0]:
import random
import glob

fileArr = []
for filename in glob.glob(os.path.join(l[0], '*.jpg')):
  fileArr.append(filename)
  
fileArr.sort()
fileArr

In [0]:
import cv2
import os
import glob
import numpy as np

l = []
for index, row in data.iterrows():
  l.append(os.path.join("/content/drive/My Drive/Dhivya_Classifier/videos/",row.colA))
print(l)

for path in l:
  fileArr = []
  for filename in glob.glob(os.path.join(path, '*.jpg')):
    fileArr.append(filename)
  fileArr.sort()
  print(fileArr)
  
  temp = []
  for img in fileArr:
    src = cv2.imread(img, cv2.IMREAD_UNCHANGED)
    src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    src = cv2.resize(src, (20, 20))
    temp.append(src)
  temp= np.subtract(np.array(temp), 128.0).astype('int8')
  savepath =  path[:-1] + ".npz"
  np.savez_compressed(savepath, temp)